# Optical Character Recognition (OCR) Topps Sticker

## Libraries and settings

In [213]:
# Libraries
import os
import re
import cv2
import pytesseract
from pytesseract import Output
import PIL
from PIL import Image
import fnmatch
import tempfile
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/Users/ivesbrunner/Documents/Studium/01_Bachelor/04_Semester/04_ScientificProgramming/euro_2024_trends/sticker_collector


## Read data from image

### Open and crop image

In [214]:
# Import the Image module from the PIL library
from PIL import Image

# Define the path to the image
image_path = './stickers/sticker_example.jpeg'

# Open the image using the path
image = Image.open(image_path)

# Get the size of the image (width and height)
width, height = image.size

# Define the dimensions for cropping
# We start from (1100, 800) and go till the image's width and height minus 500
crop_dimensions = (1100, 800, width, height-500)

# Crop the image using the defined dimensions
cropped_image = image.crop(crop_dimensions)

### Sticker class

In [215]:
class Sticker:
    def __init__(self, team, number):
        self.team = team
        self.number = number

### Read team and number sticker

In [216]:
# Convert the cropped image to text using pytesseract
# The '--psm 11' config option stands for 'Sparse text. Find as much text as possible in no particular order.'
text = pytesseract.image_to_string(cropped_image, config='--psm 11')

# Use a regular expression to replace all occurrences of multiple line breaks with a single line break
text = re.sub(r'\n+', '\n', text)

# Split the text by line breaks and get the first line as the team
team = text.split('\n')[0]

# Split the text by line breaks and get the second line as the number
number = text.split('\n')[1]

# Initalize the sticker object
sticker = Sticker(team, number)

# Print the found sticker information
print("Found sticker with the following information:")
print("Team: ", sticker.team)
print("Number: ", sticker.number)

Found sticker with the following information:
Team:  ITA
Number:  20


## Add sticker to collection if not owned yet (Postgres)

### Prepare database credentials

In [217]:
# Define the details for the database connection
host = 'localhost'  # The host server
port = '5432'  # The port to connect on
database = 'postgres'  # The database to connect to
user = 'admin'  # The user to connect as
password = 'secret'  # The user's password

### Fetch stickers

In [218]:
def fetch_stickers():
    # Establish a connection to the database using the defined details
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    
    # Create a cursor object from the connection object
    # The cursor is used to execute SQL commands
    cursor = conn.cursor()

    # Define the SQL query to select all stickers, ordered by team and number
    sql_query = "SELECT * FROM stickers ORDER BY team ASC, number ASC;"

    # Execute the SQL query using the cursor
    cursor.execute(sql_query)

    # Fetch all the results of the executed SQL query
    result = cursor.fetchall()

    # Print a message indicating the current stickers in the database
    print("Current stickers in the database:")

    # Check if the result is empty (i.e., no stickers were found in the database)
    if len(result) == 0:
        print("No stickers found in the database.")

    # If the result is not empty, print each row in the result
    for row in result:
        print(row)

    # Close the cursor and the connection to free up resources
    cursor.close()

    # Close the connection to the database
    conn.close()

    return result


# Call the fetch_stickers() function to fetch and print the stickers in the database
fetch_stickers()

Current stickers in the database:
(10, 'ESP')
(20, 'ESP')


[(10, 'ESP'), (20, 'ESP')]

### Store sticker to database

In [219]:
# Establish a connection to the database using the defined details
conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)

# Create a cursor object from the connection object
# The cursor is used to execute SQL commands
cursor = conn.cursor()

# Define the SQL query to check if the sticker exists in the database
# The %s placeholders will be replaced by the values of team and number
check_query = "SELECT * FROM stickers WHERE team = %s AND number = %s"

# Execute the SQL query using the cursor
cursor.execute(check_query, (sticker.team, sticker.number))

# Fetch all the results of the executed SQL query
result = cursor.fetchall()

# If the result is empty (i.e., the sticker does not exist in the database), insert it
if not result:
    # Define the SQL query to insert the sticker into the database
    insert_query = "INSERT INTO stickers (team, number) VALUES (%s, %s)"
    
    # Execute the SQL query using the cursor
    cursor.execute(insert_query, (sticker.team, sticker.number))
    
    # Commit the transaction to the database
    conn.commit()
    
    # Print a message indicating that the sticker has been added
    print("Sticker added to the database.")
else:
    # Print a message indicating that the sticker already exists
    print("Sticker already exists in the database.")

# Close the cursor to free up resources
cursor.close()

# Close the connection to the database
# This is important to free up resources and ensure that all changes have been committed to the database
conn.close()

Sticker added to the database.


## Create webapp to display stickers per team collected

In [223]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# Colors
colors = {
    'background': '#1f1f1f',
    'text': '#ddd'
}

# Stores stickers in dataframe
stickers = fetch_stickers()
df = pd.DataFrame(stickers, columns=['Number', 'Team'])

# Create new dataframe with count of stickers per team and rename column to 'Count'
sticker_per_team = df.groupby('Team').size().reset_index(name='Count')

# Figure
fig = px.bar(sticker_per_team, x="Team", y="Count", color="Team")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

# Layout
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Topps Sticker Collector',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'font-family': 'Arial'
        }
    ),

    html.Div(children='Get an overview of your current sticker collection', style={
        'textAlign': 'center',
        'color': colors['text'],
            'font-family': 'Arial'
    }),

    dcc.Graph(
        id='stickers-graph-1',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Current stickers in the database:
(10, 'ESP')
(20, 'ESP')
(20, 'ITA')


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [221]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.5.0
Datetime: 2024-05-30 09:53:00
Python Version: 3.10.13
-----------------------------------
